# Fabric Data Agent File Support
This notebook shows how to configure a Fabric Data Agent, ask natural language questions against lakehouse tables, and evaluate responses using the evaluation toolkit.

In [ ]:
# Install the SDK
%pip install fabric-data-agent-sdk --quiet

## Seed Sample Tables
This optional step downloads AdventureWorks sample tables and writes them to a lakehouse. Uncomment and run the sample code only when the tables are missing.

```python
# import pandas as pd
## from tqdm.auto import tqdm
## base_url = "https://synapseaisolutionsa.z13.web.core.windows.net/data/AdventureWorks"
## tables = pd.read_csv(f"{base_url}/adventureworks.csv", names=["table"])
## for table in tqdm(tables["table"], desc="Uploading to lakehouse"):
##     df = pd.read_parquet(f"{base_url}/{table}.parquet")
##     spark.createDataFrame(df).write.mode("overwrite").saveAsTable(table)
```

## Prepare Data

In [ ]:
from fabric.dataagent.client import FabricDataAgentManagement, FabricOpenAI, create_data_agent

data_agent_name = "AdvWorksDataAgent"
# Reuse the agent when it exists, otherwise create it on demand
try:
    data_agent = FabricDataAgentManagement(data_agent_name)
except Exception:
    data_agent = create_data_agent(data_agent_name)

In [ ]:
# Add a Lakehouse as the data source for the agent
lakehouse_name = "EvaluationLH"

# Supported types include: "lakehouse", "kqldatabase", "datawarehouse", or "semanticmodel"
data_agent.add_datasource(lakehouse_name, type="lakehouse")

# Retrieve the data source object (assumes one was added)
datasource = data_agent.get_datasources()[0]

# Select relevant tables from the Lakehouse (schema: dbo)
datasource.select("dbo", "dimcustomer")
datasource.select("dbo", "dimdate")
datasource.select("dbo", "dimgeography")
datasource.select("dbo", "dimproduct")
datasource.select("dbo", "dimproductcategory")
datasource.select("dbo", "dimpromotion")
datasource.select("dbo", "dimreseller")
datasource.select("dbo", "dimsalesterritory")
datasource.select("dbo", "factinternetsales")
datasource.select("dbo", "factresellersales")

# Publish the data agent
data_agent.publish()

In [ ]:
nlq = "Give me the top 100 rows of dimproduct"
fabric_client = FabricOpenAI(artifact_name=data_agent_name)
assistant = fabric_client.beta.assistants.create(model="gpt-4", instructions="You are a helpful assistant.")
thread = fabric_client.beta.threads.create()
fabric_client.beta.threads.delete(thread.id)
thread = fabric_client.beta.threads.create()
fabric_client.beta.threads.messages.create(thread_id=thread.id, role="user", content=nlq)
run = fabric_client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)
response = fabric_client.beta.threads.messages.list(thread_id=thread.id, order="desc")

In [5]:
def pretty_print(messages):
    print("# Messages")
    for m in messages:
        print(f"{m.role}: {m.content[0].text.value}")
    print()
pretty_print(response)

StatementMeta(, b4379e35-e3f0-4937-a5d6-935442a755dc, 13, Finished, Available, Finished)

# Messages
assistant: Here are the top 100 rows of the dimproduct table, as requested. You can view all this data in the file: result_0.json. The table includes columns such as ProductKey, ProductAlternateKey, ProductSubcategoryKey, EnglishProductName, StandardCost, ListPrice, and many more product attributes.

If you need this data in a specific format or have further questions about it, let me know!
user: Give me the top 100 rows of dimproduct



In [ ]:
def get_file_ids_from_messages(messages):
    file_ids = []
    for msg in response:
        for content_block in msg.content:
            if hasattr(content_block.text, "annotations"):
                for annotation in content_block.text.annotations:
                    if hasattr(annotation, "file_path") and hasattr(annotation.file_path, "file_id"):
                        file_ids.append(annotation.file_path.file_id)
    return file_ids

file_ids = get_file_ids_from_messages(response)

In [7]:
import pandas as pd

def prepare_df(fabric_client, file_id):
    file_content = fabric_client.files.content(file_id)
    payload = file_content.json()
    columns = [column["name"] for column in payload["columns"]]
    return pd.DataFrame(payload["rows"], columns=columns)

result_frames = [prepare_df(fabric_client, file_id) for file_id in file_ids]
result_frames[0].head() if result_frames else None

StatementMeta(, b4379e35-e3f0-4937-a5d6-935442a755dc, 15, Finished, Available, Finished)

,ProductKey,ProductAlternateKey,ProductSubcategoryKey,WeightUnitMeasureCode,SizeUnitMeasureCode,EnglishProductName,SpanishProductName,FrenchProductName,StandardCost,FinishedGoodsFlag,...,ChineseDescription,ArabicDescription,HebrewDescription,ThaiDescription,GermanDescription,JapaneseDescription,TurkishDescription,StartDate,EndDate,Status
0,519,SE-R908,15,None,None,ML Road Seat/Saddle,Sillín/asiento de carretera GM,Selle de vélo de route ML,17.38,True,...,橡皮减震器可避免颠簸。,مصدات مطاطية تمتص الصدمات.,פגושי גומי הסופגים חבטות.,กันชนยางกันแรงกระแทก,Stoßleisten aus Gummi zur Stoßdämpfung.,ラバー バンパーが衝撃を吸収します。,Lastik tamponlar çarpma şiddetini emer.,2013-07-01T00:00:00,None,Current
1,520,SE-R995,15,None,None,HL Road Seat/Saddle,Sillín/asiento de carretera GA,Selle de vélo de route HL,23.37,True,...,含轻型凯夫拉纤维的比赛用车座。皮革材质。,مقعد سباق خفيف الوزن مصنوع من مادة كيفلار. جلد.,מושב Kevlar קל-משקל למרוצים. עור.,เบาะนั่งเคฟลาร์สำหรับแข่ง น้ำหนักเบา ทำจากหนัง,Leichter Kevlar-Rennsattel. Leder.,競技用軽量ケブラー レザー サドル。,Hafif kevlar yarış selesi. Deri.,2013-07-01T00:00:00,None,Current
2,521,SE-T312,15,None,None,LL Touring Seat/Saddle,Sillín/asiento de paseo GB,Selle de vélo de randonnée LL,12.04,True,...,舒适且符合人体工程学的含凝胶车座。,مقعد جيلاتيني مصمم بشكل يتناسب مع حركة الجسم.,מושב ג'ל נוח בעל עיצוב ארגונומי.,เบาะนั่งบรรจุเจล นุ่มสบาย มีรูปทรงตามหลักสรีรศ...,Komfortabler Gelsattel mit ergonomischer Form.,快適なエルゴノミック形状のジェル製サドル。,"""Konforlu, ergonomik biçimli jel sele.""",2013-07-01T00:00:00,None,Current
3,522,SE-T762,15,None,None,ML Touring Seat/Saddle,Sillín/asiento de paseo GM,Selle de vélo de randonnée ML,17.38,True,...,新型设计可缓解长时间骑乘的压力。,تصميم جديد لتخفيف الضغط عند القيادة لفترات طويلة.,עיצוב חדש להקלת לחץ ברכיבות ארוכות.,ออกแบบใหม่เพื่อบรรเทาแรงกดสำหรับการขับขี่ระยะไกล,Neues Design bietet Komfort auch auf langen St...,ロング ライドのストレスを軽減する新デザイン。,Yeni tasarım uzun sürüşlerde basıncı azaltır.,2013-07-01T00:00:00,None,Current
4,523,SE-T924,15,None,None,HL Touring Seat/Saddle,Sillín/asiento de paseo GA,Selle de vélo de randonnée HL,23.37,True,...,精致的车架，令您骑乘更舒适。,هيكل خارجي تم تصميمه لتوفير مزيد من الراحة أثن...,מעטפת קעורה לרכיבה נוחה יותר.,เปลือกพิเศษสำหรับการขับขี่ที่แสนสบาย,Ausgeschnittene Form für komfortableres Fahren.,より快適な乗り心地のためのカット アウト シェル。,Daha konforlu bir sürüş için özel sele kaplaması.,2013-07-01T00:00:00,None,Current


## Evaluation

In [ ]:
import pandas as pd

# Create DataFrame with "question,expected_answer". Please update the questions and expected_answers as per the requirement.
df = pd.DataFrame(columns=["question", "expected_answer"],
                  data=[
                    ["What were our total sales in 2014?", "45,694.7"],
                    ["What is the most sold product?", "Mountain-200 Black, 42"],
                    ["What are the most expensive items that have never been sold?", "Road-450 Red, 60"],
                ])

# You can also oad from input CSV file with data in format "question,expected_answer"
# input_file_path = "/lakehouse/default/Files/Data/Input/groundtruth.csv"
# df = pd.read_csv(input_file_path)

In [ ]:
from fabric.dataagent.evaluation import evaluate_data_agent

workspace_name = None
table_name = "demo_evaluation_output"
data_agent_stage = "sandbox"
max_workers = 4
num_query_repeats = 3

In [10]:
evaluation_id = evaluate_data_agent(
    df,
    data_agent_name,
    workspace_name=workspace_name,
    table_name=table_name,
    data_agent_stage=data_agent_stage,
    max_workers=max_workers,
    num_query_repeats=num_query_repeats,)

print(f"Unique Id for the current evaluation run: {evaluation_id}")

StatementMeta(, b4379e35-e3f0-4937-a5d6-935442a755dc, 18, Finished, Available, Finished)

**🔗Failed Thread(s):**
- [thread_qzwFCQEuuGYo9v69OZ9s1z8j](https://msit.fabric.microsoft.com/workloads/de-ds/dataagents/2d2c7f00-eb24-40ad-965d-0eb812d2fd19/externalThread?debug.aiSkillThreadIdOverride=thread_qzwFCQEuuGYo9v69OZ9s1z8j&debug.aiSkillViewPublishedOverride=0)
- [thread_oJv4iCqNrxwhr7zaSSektWaJ](https://msit.fabric.microsoft.com/workloads/de-ds/dataagents/2d2c7f00-eb24-40ad-965d-0eb812d2fd19/externalThread?debug.aiSkillThreadIdOverride=thread_oJv4iCqNrxwhr7zaSSektWaJ&debug.aiSkillViewPublishedOverride=0)
- [thread_aLITOqeiJ2Pyees0oIsx7FSp](https://msit.fabric.microsoft.com/workloads/de-ds/dataagents/2d2c7f00-eb24-40ad-965d-0eb812d2fd19/externalThread?debug.aiSkillThreadIdOverride=thread_aLITOqeiJ2Pyees0oIsx7FSp&debug.aiSkillViewPublishedOverride=0)

100%|██████████| 9/9 [01:11<00:00,  7.94s/it]


Unique Id for the current evaluation run: 54593c15-7c10-42ec-8bd9-d6f247f35cfe


In [11]:
# Import the function to retrieve evaluation summaries
from fabric.dataagent.evaluation import get_evaluation_summary

# Retrieve the summary of the evaluation results using the specified table name
# This returns a DataFrame with aggregated metrics like counts of true/false/unclear responses
eval_summary_df = get_evaluation_summary(table_name, verbose=True)

StatementMeta(, b4379e35-e3f0-4937-a5d6-935442a755dc, 19, Finished, Available, Finished)

index,evaluation_id,T,F,?,%
0,54593c15-7c10-42ec-8bd9-d6f247f35cfe,6,3,0,66.7
1,8b79e754-2911-4767-9d46-6aa7494fbd00,5,4,0,55.6
2,cb968aa7-342d-44cb-b8fc-616d79adfee3,6,3,0,66.7
3,d16309f5-2a89-46fb-a692-3dccf9fb4bb0,2,1,0,66.7
4,d90020c0-7a84-4697-8f63-774b31222d65,1,2,0,33.3


In [ ]:
import pandas as pd

with pd.option_context('display.max_colwidth', None):
  display(eval_summary_df)

In [14]:
from fabric.dataagent.evaluation import get_evaluation_details

get_all_rows = True
verbose = True
eval_details_df = get_evaluation_details(
    evaluation_id,
    table_name,
    get_all_rows=get_all_rows,
    verbose=verbose,)

StatementMeta(, b4379e35-e3f0-4937-a5d6-935442a755dc, 22, Finished, Available, Finished)

index,question,expected_answer,actual_answer,evaluation_judgement,thread_url
0,What are the most expensive items that have never been sold?,"Road-450 Red, 60","Here are the most expensive items that have never been sold: File: result_0.json | ProductKey | EnglishProductName | ListPrice | |:-----------|:--------------------------|:----------| | 315 | Road-450 Red, 58 | 1457.99 | | 316 | Road-450 Red, 60 | 1457.99 | | 317 | Road-450 Red, 44 | 1457.99 | | 318 | Road-450 Red, 48 | 1457.99 | | 319 | Road-450 Red, 52 | 1457.99 | | 240 | HL Road Frame - Red, 62 | 1431.50 | | 440 | HL Road Frame - Black, 44 | 1431.50 | | 442 | HL Road Frame - Black, 48 | 1431.50 | | 444 | HL Road Frame - Black, 52 | 1431.50 | | 438 | HL Road Frame - Black, 62 | 1431.50 | These items have the highest list prices and have never been sold. If you need the complete list or more details, let me know! | ProductKey | EnglishProductName | ListPrice | |-------------:|:-------------------------------|------------:| | 315 | Road-450 Red, 58 | 1457.99 | | 316 | Road-450 Red, 60 | 1457.99 | | 317 | Road-450 Red, 44 | 1457.99 | | 318 | Road-450 Red, 48 | 1457.99 | | 319 | Road-450 Red, 52 | 1457.99 | | 240 | HL Road Frame - Red, 62 | 1431.5 | | 440 | HL Road Frame - Black, 44 | 1431.5 | | 442 | HL Road Frame - Black, 48 | 1431.5 | | 444 | HL Road Frame - Black, 52 | 1431.5 | | 438 | HL Road Frame - Black, 62 | 1431.5 | | 252 | HL Road Frame - Red, 56 | 1431.5 | | 249 | HL Road Frame - Red, 52 | 1431.5 | | 243 | HL Road Frame - Red, 44 | 1431.5 | | 246 | HL Road Frame - Red, 48 | 1431.5 | | 292 | HL Mountain Frame - Silver, 48 | 1364.5 | | 309 | HL Mountain Frame - Silver, 38 | 1364.5 | | 295 | HL Mountain Frame - Silver, 46 | 1364.5 | | 291 | HL Mountain Frame - Silver, 44 | 1364.5 | | 290 | HL Mountain Frame - Silver, 42 | 1364.5 | | 298 | HL Mountain Frame - Black, 42 | 1349.6 | | 299 | HL Mountain Frame - Black, 44 | 1349.6 | | 303 | HL Mountain Frame - Black, 46 | 1349.6 | | 306 | HL Mountain Frame - Black, 38 | 1349.6 | | 300 | HL Mountain Frame - Black, 48 | 1349.6 | | 439 | HL Road Frame - Black, 44 | 1301.36 |",True,thread_Df8s7iICdxhR5La3Q4MHmLrF
1,What are the most expensive items that have never been sold?,"Road-450 Red, 60","Here are the most expensive items that have never been sold, along with their prices (from file result_0.json): | ProductName | ListPrice | |:--------------------------|:----------| | Road-450 Red, 52 | 1457.99 | | Road-450 Red, 48 | 1457.99 | | Road-450 Red, 44 | 1457.99 | | Road-450 Red, 60 | 1457.99 | | Road-450 Red, 58 | 1457.99 | | HL Road Frame - Black, 52 | 1431.50 | | HL Road Frame - Black, 48 | 1431.50 | | HL Road Frame - Black, 44 | 1431.50 | | HL Road Frame - Black, 62 | 1431.50 | | HL Road Frame - Red, 56 | 1431.50 | If you need more results or further details, let me know! | ProductName | ListPrice | |:-------------------------------|------------:| | Road-450 Red, 52 | 1457.99 | | Road-450 Red, 48 | 1457.99 | | Road-450 Red, 44 | 1457.99 | | Road-450 Red, 60 | 1457.99 | | Road-450 Red, 58 | 1457.99 | | HL Road Frame - Black, 52 | 1431.5 | | HL Road Frame - Black, 48 | 1431.5 | | HL Road Frame - Black, 44 | 1431.5 | | HL Road Frame - Black, 62 | 1431.5 | | HL Road Frame - Red, 56 | 1431.5 | | HL Road Frame - Red, 52 | 1431.5 | | HL Road Frame - Red, 48 | 1431.5 | | HL Road Frame - Red, 44 | 1431.5 | | HL Road Frame - Red, 62 | 1431.5 | | HL Mountain Frame - Silver, 38 | 1364.5 | | HL Mountain Frame - Silver, 48 | 1364.5 | | HL Mountain Frame - Silver, 42 | 1364.5 | | HL Mountain Frame - Silver, 44 | 1364.5 | | HL Mountain Frame - Silver, 46 | 1364.5 | | HL Mountain Frame - Black, 38 | 1349.6 | | HL Mountain Frame - Black, 48 | 1349.6 | | HL Mountain Frame - Black, 46 | 1349.6 | | HL Mountain Frame - Black, 44 | 1349.6 | | HL Mountain Frame - Black, 42 | 1349.6 | | HL Road Frame - Black, 52 | 1301.36 |",True,thread_a5dSLKZh7mA4NLUf4Xv0DMP1
2,What are the most expensive items that have never been sold?,"Road-450 Red, 60","

In [15]:
from fabric.dataagent.evaluation import evaluate_data_agent

# Define a custom prompt to evaluate whether the agent's actual response matches the expected answer.
# The prompt should include placeholders: {query}, {expected_answer}
critic_prompt = """
        Given the following query, expected answer, and actual answer from previous query, please determine if the actual answer is equivalent to expected answer. If they are equivalent, respond with 'yes'.

        Query: {query}

        Expected Answer:
        {expected_answer}

        Is the actual answer equivalent to the expected answer?
        """

# Evaluate the Data Agent using the custom critic prompt
# Returns a unique evaluation ID for tracking and analysis
evaluation_id_critic = evaluate_data_agent(
    df,
    data_agent_name,
    critic_prompt=critic_prompt,
    table_name=table_name,
    data_agent_stage="sandbox"
)

# Retrieve the summary of this evaluation run
eval_summary_df_critic = get_evaluation_summary(table_name)

# Display the unique ID for reference
print(f"Unique Id for the current evaluation run: {evaluation_id_critic}")

eval_summary_df_critic

StatementMeta(, b4379e35-e3f0-4937-a5d6-935442a755dc, 23, Finished, Available, Finished)

**🔗Failed Thread(s):**
- [thread_0BetBcs4J1UDYHYsUmkfO3oX](https://msit.fabric.microsoft.com/workloads/de-ds/dataagents/2d2c7f00-eb24-40ad-965d-0eb812d2fd19/externalThread?debug.aiSkillThreadIdOverride=thread_0BetBcs4J1UDYHYsUmkfO3oX&debug.aiSkillViewPublishedOverride=0)

100%|██████████| 3/3 [00:24<00:00,  8.09s/it]


Unique Id for the current evaluation run: cae6f857-8e56-4c6c-aa67-f4b57a9a146a


index,evaluation_id,T,F,?,%
0,54593c15-7c10-42ec-8bd9-d6f247f35cfe,6,3,0,66.7
1,8b79e754-2911-4767-9d46-6aa7494fbd00,5,4,0,55.6
2,cae6f857-8e56-4c6c-aa67-f4b57a9a146a,2,1,0,66.7
3,cb968aa7-342d-44cb-b8fc-616d79adfee3,6,3,0,66.7
4,d16309f5-2a89-46fb-a692-3dccf9fb4bb0,2,1,0,66.7
5,d90020c0-7a84-4697-8f63-774b31222d65,1,2,0,33.3
